<a href="https://colab.research.google.com/github/raphaelp-silva/deep_learning_com_pytorch_e_python/blob/main/Projeto_7_Classifica%C3%A7%C3%A3o_multiclasse_iris_com_valida%C3%A7%C3%A3o_cruzada_com_dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto 7: Classificação multiclasse iris com validação cruzada

## Etapa 1: Importação das bibliotecas

In [1]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install --upgrade skorch

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
import pandas as pd
import torch.nn as nn
from skorch import NeuralNetClassifier
import torch
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
torch.__version__

'2.6.0+cu118'

## Etapa 2: Base de dados

In [3]:
torch.manual_seed(123)

In [4]:
base = pd.read_csv('iris.csv')
previsores = base.iloc[:, 0:4].values
classe = base.iloc[:, 4].values

In [5]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
classe = encoder.fit_transform(classe)

In [6]:
previsores = previsores.astype('float32')
classe = classe.astype('int64')

In [7]:
type(previsores)

numpy.ndarray

In [8]:
previsores.shape

(150, 4)

In [8]:
classe.shape

(150,)

## Etapa 3: Construção do modelo

In [9]:
class ClassificadorTorch(nn.Module):
    def __init__(self, activation, neurons, initializer, dropout):
        super().__init__()

        # 4 -> 16 -> 16 -> 16 -> 3  // estrutura da rede neural
        self.dense0 = nn.Linear(4, neurons)
        initializer(self.dense0.weight)
        self.activation0 = activation

        self.dense1 = nn.Linear(neurons, neurons)
        initializer(self.dense1.weight)
        self.activation1 = activation

        self.dense2 = nn.Linear(neurons, neurons)
        initializer(self.dense2.weight)
        self.activation2 = activation

        self.dense3 = nn.Linear(neurons, 3)

        self.dropout = nn.Dropout(dropout)


    def forward(self, X):
        X = self.dense0(X)
        X = self.activation0(X)
        X = self.dropout(X)
        X = self.dense1(X)
        X = self.activation1(X)
        X = self.dropout(X)
        X = self.dense2(X)
        X = self.activation2(X)
        X = self.dropout(X)
        X = self.dense3(X)
        return X


In [10]:
classificador_sklearn = NeuralNetClassifier(module = ClassificadorTorch,
                                            criterion = torch.nn.CrossEntropyLoss,
                                            train_split = False)

## Tunning dos parâmetros:

In [71]:
# definindo os parâmetros que serão testados na rede para encontrarmos o que melhor performou nos testes:

params = {'batch_size' : [10],
          'max_epochs': [500],
          'optimizer': [torch.optim.Adam], # testar torch.optim.SGD
          'criterion': [torch.nn.CrossEntropyLoss], # testar torch.nn.HingeEmbeddingLoss
          'module__activation': [F.relu], # testar F.tanh, F.sigmoid
          'module__neurons': [16], # testar 8 , 16
          'module__initializer' : [torch.nn.init.uniform_], # testar , torch.nn.init.normal_
          'module__dropout' : [0.2]} # testar 0.3

In [72]:
params

{'batch_size': [10],
 'max_epochs': [500],
 'optimizer': [torch.optim.adam.Adam],
 'criterion': [torch.nn.modules.loss.CrossEntropyLoss],
 'module__activation': [<function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>],
 'module__neurons': [16],
 'module__initializer': [<function torch.nn.init.uniform_(tensor: torch.Tensor, a: float = 0.0, b: float = 1.0, generator: Optional[torch._C.Generator] = None) -> torch.Tensor>],
 'module__dropout': [0.2]}

In [73]:
grid_search = GridSearchCV(estimator = classificador_sklearn,
                           param_grid = params,
                           scoring = 'accuracy',
                           cv = 3)

In [74]:
grid_search = grid_search.fit(previsores, classe)

  epoch    train_loss     dur
-------  ------------  ------
      1      405.6385  0.0225
      2       60.9651  0.0296
      3       81.3417  0.0276
      4       48.8649  0.0245
      5       22.6361  0.0383
      6       17.2298  0.0222
      7       11.2621  0.0288
      8        8.8168  0.0266
      9        7.5593  0.0234
     10        7.2168  0.0249
     11        4.3693  0.0242
     12        4.4366  0.0239
     13        3.3002  0.0251
     14        2.7040  0.0275
     15        2.2423  0.0241
     16        2.3549  0.0281
     17        2.0326  0.0239
     18        1.5459  0.0289
     19        1.4965  0.0268
     20        1.3564  0.0269
     21        1.2436  0.0268
     22        0.9709  0.0246
     23        0.9625  0.0266
     24        1.0395  0.0251
     25        0.9438  0.0245
     26        0.7985  0.0241
     27        0.9217  0.0267
     28        0.9041  0.0263
     29        0.9586  0.0277
     30        0.8343  0.0302
     31        0.9067  0.0220
     32   

In [75]:
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_

In [76]:
melhores_parametros

{'batch_size': 10,
 'criterion': torch.nn.modules.loss.CrossEntropyLoss,
 'max_epochs': 500,
 'module__activation': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
 'module__dropout': 0.2,
 'module__initializer': <function torch.nn.init.uniform_(tensor: torch.Tensor, a: float = 0.0, b: float = 1.0, generator: Optional[torch._C.Generator] = None) -> torch.Tensor>,
 'module__neurons': 16,
 'optimizer': torch.optim.adam.Adam}

In [77]:
melhor_precisao

0.98